<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf


    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance


    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-07-24 14:57:15
-------------------
qualified stocks: 96
with latest results: 31
still star stocks: 20
-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  10.00 K
Current:  1.36 C
-------------------
Today PnL: -69.09 K (-0.5%)
Current PnL: -14.73 L (-10.26%)
CY Booked + Current PnL: -6.65 L (-4.64%)
-------------------
Total profit:  3.57 L
Total loss:  -18.30 L
-------------------
Total Booked + Current PnL: 20.33 L (17.44%)
Total Booked PnL: 35.06 L (30.08%)
Curr Year Booked PnL: 8.08 L (5.92%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.09 C
Est FTT PnL: 72.94 L (53.46%)
Deployed:  1.17 C
Current:  1.36 C
CAGR/XIRR %: 10.32%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
92,VIPIND,488.80,489.00,-222.20,H-SC,10.57,88120.0,-7196.0,7235.0,-1.77,-7.55,8.21,0.04,161.0,-0.99,0.65,73.81,OX40N,NTT,MISC
39,HINDALCO,651.95,761.55,-19.90,H-LC,5.29,110776.0,6464.0,11067.0,-0.44,6.20,9.99,16.81,9.0,0.58,0.81,23.24,X5K,ATH,METALS
70,SBIN,760.30,863.00,-10.56,M-LC,4.82,216002.0,14522.0,12701.0,-0.68,7.21,5.88,13.51,58.0,1.14,1.58,20.71,XY25,NTT,BANKS
4,APOLLOHOSP,6904.43,8285.00,-18.84,H-LC,6.67,110558.0,6992.0,13720.0,-0.04,6.75,12.41,20.00,14.0,0.51,0.81,21.77,X40N,BTT,HEALTHCARE
17,BRITANNIA,5190.70,6446.05,13.77,H-LC,10.03,102051.0,8618.0,13981.0,-0.87,9.22,13.70,24.18,32.0,0.62,0.75,23.92,XY25,ATH,FMCG


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,HEROMOTOCO,4311.81,5949.07,-6.67,H-MC,2.46,150073.0,-840.0,58138.0,-0.25,-0.56,38.74,37.97,87.0,-0.01,1.1,24.61,AR,ATH,AUTO


In [11]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,CIPLA,1492.70,1795.00,-22.84,H-LC,0.77,203294.0,-1206.0,42631.0,0.71,-0.59,20.97,20.25,11.0,-0.03,1.49,8.09,X40N,BTT,PHARMA
10,BAJAJHFL,122.26,152.00,-5.28,H-LC,2.03,196406.0,-4100.0,52872.0,-2.05,-2.05,26.92,24.33,23.0,-0.08,1.44,13.26,X40N,BTT,FINANCE
40,HINDUNILVR,2413.81,2723.00,-12.82,H-LC,2.46,272933.0,2586.0,32042.0,-0.55,0.96,11.74,12.81,16.0,0.08,2.00,14.15,XY25,NTT,FMCG
56,LTIM,5564.16,7230.20,-2.76,H-LC,2.70,187290.0,-13020.0,73006.0,-1.80,-6.50,38.98,29.94,28.0,-0.18,1.37,29.39,X200,ATH,IT
41,HINDZINC,471.44,730.22,18.37,H-LC,2.89,193575.0,-11501.0,124062.0,-0.41,-5.61,64.09,54.89,3.0,-0.09,1.42,17.14,X5K,ATH,METALS


In [12]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
16,BLUSPRING,226.45,94.95,NaN,NaN,5.08,20571.0,-29022.0,224.0,6.13,-58.52,1.09,-58.07,201.0,-129.56,0.15,37.95,XY24,ATH,MISC
7,ATULAUTO,624.44,844.00,3440.54,M-SC,11.23,110448.0,-36920.0,88734.0,3.61,-25.05,80.34,35.16,233.0,-0.42,0.81,12.81,XY24,NTT,AUTO
54,LALPATHLAB,2838.53,3545.00,-26.32,H-SC,1.84,209719.0,16699.0,31332.0,2.22,8.65,14.94,24.89,122.0,0.53,1.54,33.77,X40N,NTT,HEALTHCARE
55,LAOPALA,369.40,464.00,138.65,H-SC,6.85,75730.0,-25116.0,50944.0,2.12,-24.91,67.27,25.61,133.0,-0.49,0.56,36.58,AR,NTT,CERAMICS
84,TRIDENT,37.35,48.00,23.89,M-SC,4.66,77928.0,-14326.0,40632.0,1.77,-15.53,52.14,28.51,239.0,-0.35,0.57,33.35,XR,NTT,TEXTILES


In [13]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,NESTLEIND,2268.60,2755.00,-14.12,H-LC,5.75,272282.0,6856.0,50045.0,-5.11,2.58,18.38,21.44,19.0,0.14,2.00,9.40,XY25,NTT,FMCG
28,DIGITIDE,188.38,267.99,NaN,NaN,0.69,56382.0,15127.0,2306.0,-3.72,36.67,4.09,42.26,197.0,6.56,0.41,32.74,XY24,ATH,IT
76,STARHEALTH,551.56,761.00,-4.95,M-SC,3.25,155673.0,-44543.0,120569.0,-3.07,-22.25,77.45,37.97,220.0,-0.37,1.14,24.99,XY24,NTT,INSURANCE
2,ACC,2579.05,3906.00,-35.90,M-MC,4.92,138131.0,-50140.0,147013.0,-2.96,-26.63,106.43,51.45,173.0,-0.34,1.01,5.02,XY24,BTT,CEMENT
86,UJJIVANSFB,52.77,53.00,42.36,M-SC,23.83,123606.0,-18873.0,19493.0,-2.64,-13.25,15.77,0.44,249.0,-0.97,0.91,45.89,OX40N,NTT,BANKS


In [14]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
92,VIPIND,488.80,489.0,-222.20,H-SC,10.57,88120.0,-7196.0,7235.0,-1.77,-7.55,8.21,0.04,161.0,-0.99,0.65,73.81,OX40N,NTT,MISC
86,UJJIVANSFB,52.77,53.0,42.36,M-SC,23.83,123606.0,-18873.0,19493.0,-2.64,-13.25,15.77,0.44,249.0,-0.97,0.91,45.89,OX40N,NTT,BANKS
78,SYMPHONY,1306.42,1306.0,-30.89,M-SC,10.52,147244.0,-23897.0,23839.0,-0.70,-13.96,16.19,-0.03,194.0,-1.00,1.08,6.55,OX40N,NTT,DURABLES
80,TATAELXSI,7332.28,7332.0,-16.97,H-MC,1.23,87150.0,-15502.0,15495.0,-0.17,-15.10,17.78,-0.00,101.0,-1.00,0.64,32.48,OX40N,NTT,IT
46,INDIGOPNTS,1407.73,1408.0,123.77,H-SC,7.96,146618.0,-27941.0,27975.0,-1.50,-16.01,19.08,0.02,129.0,-1.00,1.08,27.14,OX40N,NTT,PAINTS


In [15]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
91,VBL,492.64,672.28,-13.77,H-LC,13.28,261602.0,-3438.0,100089.0,-0.31,-1.30,38.26,36.46,1.0,-0.03,1.92,11.72,X40N,ATH,FMCG
32,ENRIN,1377.95,3176.70,NaN,NaN,2.39,75180.0,40731.0,4240.0,-0.11,118.24,5.64,130.54,2.0,9.61,0.55,14.96,AR,ATH,ELECTRICAL
41,HINDZINC,471.44,730.22,18.37,H-LC,2.89,193575.0,-11501.0,124062.0,-0.41,-5.61,64.09,54.89,3.0,-0.09,1.42,17.14,X5K,ATH,METALS
82,TCS,3794.03,4998.00,-24.49,H-LC,11.77,245762.0,-50172.0,144090.0,-0.89,-16.95,58.63,31.73,4.0,-0.35,1.80,0.00,X40,BTT,IT
48,INFY,1461.46,2275.00,-12.06,H-LC,7.07,281836.0,20235.0,125389.0,0.23,7.73,44.49,55.67,6.0,0.16,2.07,14.31,X40,BTT,IT


In [16]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
28,DIGITIDE,188.38,267.99,NaN,NaN,0.69,56382.0,15127.0,2306.0,-3.72,36.67,4.09,42.26,197.0,6.56,0.41,32.74,XY24,ATH,IT
58,MEDANTA,1087.93,1486.00,19.85,H-SC,9.17,153630.0,27430.0,18743.0,0.89,21.74,12.20,36.59,151.0,1.46,1.13,36.97,XY24,NTT,HEALTHCARE
32,ENRIN,1377.95,3176.70,NaN,NaN,2.39,75180.0,40731.0,4240.0,-0.11,118.24,5.64,130.54,2.0,9.61,0.55,14.96,AR,ATH,ELECTRICAL


In [17]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
67,SAIL,130.64,228.00,95.22,M-MC,14.14,234192.0,9230.0,158431.0,0.90,4.10,67.65,74.53,193.0,0.06,1.72,35.97,XY24,BTT,STEEL
53,KPIGREEN,497.21,731.64,31.49,H-SC,12.50,134505.0,9208.0,49874.0,0.77,7.35,37.08,47.15,144.0,0.18,0.99,68.27,MH,ATH,POWER
58,MEDANTA,1087.93,1486.00,19.85,H-SC,9.17,153630.0,27430.0,18743.0,0.89,21.74,12.20,36.59,151.0,1.46,1.13,36.97,XY24,NTT,HEALTHCARE
20,CAMS,3643.00,5226.82,4.54,H-SC,1.93,117404.0,15400.0,28952.0,-1.16,15.10,24.66,43.48,121.0,0.53,0.86,34.41,X40N,ATH,MISC
95,WIPRO,243.46,420.00,-5.58,M-LC,5.65,161696.0,10751.0,98699.0,-0.23,7.12,61.04,72.51,55.0,0.11,1.19,11.38,XR,NTT,IT


In [18]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,ENRIN,1377.95,3176.70,NaN,NaN,2.39,75180.0,40731.0,4240.0,-0.11,118.24,5.64,130.54,2.0,9.61,0.55,14.96,AR,ATH,ELECTRICAL
94,WHIRLPOOL,1219.98,2270.00,-36.92,M-SC,9.64,106125.0,14626.0,64121.0,-2.10,15.99,60.42,86.07,207.0,0.23,0.78,53.52,XR,NTT,DURABLES
59,NATIONALUM,189.63,247.44,-47.89,H-MC,3.88,105375.0,4871.0,25764.0,0.37,4.85,24.45,30.49,82.0,0.19,0.77,38.82,MH,ATH,METALS
53,KPIGREEN,497.21,731.64,31.49,H-SC,12.50,134505.0,9208.0,49874.0,0.77,7.35,37.08,47.15,144.0,0.18,0.99,68.27,MH,ATH,POWER
45,INDIAMART,2327.09,4850.92,-46.77,H-SC,12.87,138717.0,15381.0,118381.0,1.02,12.47,85.34,108.45,118.0,0.13,1.02,37.61,AR,ATH,MISC


In [19]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
16,BLUSPRING,226.45,94.95,NaN,NaN,5.08,20571.0,-29022.0,224.0,6.13,-58.52,1.09,-58.07,201.0,-129.56,0.15,37.95,XY24,ATH,MISC
28,DIGITIDE,188.38,267.99,NaN,NaN,0.69,56382.0,15127.0,2306.0,-3.72,36.67,4.09,42.26,197.0,6.56,0.41,32.74,XY24,ATH,IT
74,SIS,477.00,477.00,2315.54,M-SC,5.58,64991.0,-16099.0,16098.0,0.09,-19.85,24.77,0.00,236.0,-1.00,0.48,31.71,OX40N,NTT,MISC
24,COFFEEDAY,59.14,80.00,-51.71,L-SC,21.40,70925.0,-42624.0,82677.0,-0.99,-37.54,116.57,35.27,266.0,-0.52,0.52,73.59,XR,NTT,HOTELS
84,TRIDENT,37.35,48.00,23.89,M-SC,4.66,77928.0,-14326.0,40632.0,1.77,-15.53,52.14,28.51,239.0,-0.35,0.57,33.35,XR,NTT,TEXTILES


In [20]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
16,BLUSPRING,226.45,94.95,NaN,NaN,5.08,20571.0,-29022.0,224.0,6.13,-58.52,1.09,-58.07,201.0,-129.56,0.15,37.95,XY24,ATH,MISC
33,GREENPANEL,375.16,537.00,248.84,M-SC,11.07,155770.0,-27308.0,106282.0,1.33,-14.92,68.23,43.14,223.0,-0.26,1.14,46.52,XY24,NTT,MISC
64,RELAXO,902.64,1176.00,-29.99,H-SC,15.83,63270.0,-52268.0,87256.0,-0.33,-45.24,137.91,30.28,132.0,-0.60,0.46,23.61,X40N,NTT,FOOTWEAR
58,MEDANTA,1087.93,1486.00,19.85,H-SC,9.17,153630.0,27430.0,18743.0,0.89,21.74,12.20,36.59,151.0,1.46,1.13,36.97,XY24,NTT,HEALTHCARE
29,DIXON,14367.00,18940.15,-27.05,H-MC,3.19,134328.0,19392.0,17194.0,1.42,16.87,12.80,31.83,83.0,1.13,0.99,53.96,X40N,ATH,IT


In [21]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.52
1,25,41.08
2,50,69.96


In [22]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    41.94
LC    34.92
MC    22.03
Name: CurrAlloc%, dtype: float64

In [23]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     29.97
X40N     14.42
X40      12.46
XY25     10.72
XR        9.25
OX40N     8.39
AR        8.27
X5K       2.23
MH        1.76
X200      1.37
SR        1.16
Name: CurrAlloc%, dtype: float64

In [24]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    28.05
H-SC    25.55
H-MC    18.95
M-SC    14.96
M-LC     5.83
M-MC     2.73
L-SC     1.43
L-LC     1.04
L-MC     0.35
Name: CurrAlloc%, dtype: float64

In [25]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,13.98,-2.70,37.05
IT,11.86,-12.33,67.68
FINANCE,8.44,-13.32,54.44
BANKS,7.69,-8.59,52.91
MISC,6.05,-16.75,71.28
PAINTS,5.60,-14.37,36.91
ELECTRICAL,5.39,-0.08,47.51
HEALTHCARE,4.53,2.07,23.94
AUTO,4.40,-12.67,61.68


In [26]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2629972.0,25
XR,1033618.0,13
AR,1017645.0,10
X40,754743.0,10
X40N,584511.0,12
XY25,476492.0,8
OX40N,329268.0,11
SR,184478.0,2
X5K,135129.0,2


In [27]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2341619.0,26
M-SC,1590359.0,20
H-LC,1356865.0,21
H-MC,1121063.0,15
M-MC,305444.0,2
M-LC,266026.0,4
L-SC,205044.0,3
L-MC,54259.0,1
L-LC,47051.0,1


In [28]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,,sum,count
Conviction,Criteria,,
H-SC,XY24,918085.0,7
M-SC,XY24,765456.0,7
H-SC,AR,505589.0,3
H-LC,X40,486915.0,6
H-MC,XY24,434309.0,4
H-SC,XR,338736.0,4
H-LC,X40N,307785.0,6
M-MC,XY24,305444.0,2
M-SC,XR,279606.0,4


In [29]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 27.0 seconds
